In [8]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy import signal
import sys

In [9]:
def GetRMSE(x2, y2, x1, y1):
    from scipy.spatial.distance import pdist

    er = []
    for idx in range(len(x2)):
        X = np.asarray([[x1[idx], y1[idx]], 
                         [x2[idx], y2[idx]]])
        temp_er = pdist(X,metric = 'euclidean')
        er.append(temp_er[0])
    er = np.asarray(er)
    return(er)

In [10]:
# parameters used to filter data

cutoff = 24

interpol_order = 3 # order for polynomial interpolation

# win_gauss = signal.gaussian(10,3) # gaussian kernal for smoothening interpolated data

# parameters for the savitzky-golay filter
savgol_win = 11
savgol_polyorder = 3

# window_length=7, polyorder=2,

# ROLLING_WINDOW = [11, 21, 31][0]

### get the r from the manually tracked data

In [11]:
direc = r"../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/CleanedUp/"
visitnum = ['FirstVisit/', 'Later7thVisit/','LaterVisit/', 'LastVisit/']

for visit in visitnum[0:1]:
    path = os.path.join(direc, visit)
    trackslist = glob.glob(path + '*.csv')
    
#     pathFrame = os.path.join('../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/',
#                             visit)
#     frameReference = pd.read_csv(pathFrame + "frameNumberReference.csv")

In [12]:
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [13]:
outpath = os.path.join('../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/FinalTracks/', visit)
print(outpath)

if not os.path.exists(outpath):
    os.mkdir(outpath)

../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/FinalTracks/FirstVisit/


In [14]:
for data in trackslist:
#     data = trackslist[0]
    name = os.path.basename(data)[:-4]
    print('working on ' + name)
          
    file = pd.read_csv(data)
    
    # get the frameNum that is relevant
    segments = name.split("_")
    moth = segments[0] + '_' + segments[1]
    
    
    strt = file.first_valid_index()
    stop = file.last_valid_index()
    
    x = file[strt:stop].pt1_cam1_X.values
    y = file[strt:stop].pt1_cam1_Y.values
    
    name = [n for n in circ_parameters.name if n + '_' in data][0]

    circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
    circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
    circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values
    
    filtered_x = x 
    filtered_y = y 
    filtered_r = np.linalg.norm([filtered_x-circ_x, filtered_y-circ_y], axis = 0)
    filtered_r = filtered_r/circ_radii
    trajectory = pd.DataFrame([filtered_x, filtered_y, filtered_r]).T
    trajectory.columns = ['x', 'y', 'r']
    
    trajectory.to_csv(outpath + name + '_' + visit[:-1] + '.csv')

working on c-10_m11_FirstVisit_Edited_xypts
working on c-10_m13_FirstVisit_Edited_xypts
working on c-10_m15_FirstVisit_Edited_xypts
working on c-1_m13_FirstVisit_Edited_xypts
working on c-1_m14_FirstVisit_Edited_xypts
working on c-1_m17_FirstVisit_Edited_xypts
working on c-2_m20_FirstVisit_Edited_xypts
working on c-2_m23_FirstVisit_Edited_xypts
working on c-3_m10_FirstVisit_Edited_xypts
